In [2]:
from google.colab import files
uploaded = files.upload()

Saving final_data_for_model_training.zip to final_data_for_model_training.zip


In [3]:
import zipfile
import os

# Assuming you uploaded 'my_folder.zip'
with zipfile.ZipFile('final_data_for_model_training.zip', 'r') as zip_ref:
    zip_ref.extractall('my_folder')

# Check contents
print(os.listdir('my_folder'))

['final_data_for_model_training.csv']


In [5]:
import pandas as pd
import numpy as np
import sklearn

In [8]:
dataset = pd.read_csv('my_folder/final_data_for_model_training.csv')

In [9]:
ques_df = dataset[['question1','question2']]
ques_df.head()

,question1,question2
0,what is the best marketing automation tool for...,what is the best marketing automation tool for...
1,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...
2,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...
3,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...
4,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy


In [10]:
final_df = dataset.drop(columns=['question1','question2'])
print(final_df.shape)
final_df.head()

(29999, 23)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,1,75,76,13,13,12,26,0.46,0.874989,0.874989,...,0.923070,1,1,0,13.0,0.855263,99,99,99,99
1,0,48,56,13,16,8,24,0.33,0.666644,0.499988,...,0.466664,1,1,3,13.5,0.224490,69,67,65,74
2,0,104,119,28,21,4,38,0.11,0.000000,0.000000,...,0.115384,0,0,6,23.0,0.047619,26,29,34,43
3,0,58,145,14,32,1,34,0.03,0.000000,0.000000,...,0.000000,0,0,17,21.5,0.050847,29,41,23,30
4,0,34,49,5,9,3,13,0.23,0.749981,0.599988,...,0.333330,1,0,4,7.0,0.542857,55,70,48,69


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [12]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(29999, 6000)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [14]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train,y_train)



RandomForestClassifier()

In [19]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train,y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [21]:
# for xgboost model
y_pred1 = xgb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
confusion_matrix(y_test,y_pred1)


0.7543333333333333


array([[3102,  713],
       [ 761, 1424]])

In [18]:
# for random forest model
y_pred = rf.predict(X_test)
print(accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)


0.7491666666666666


array([[3091,  724],
       [ 781, 1404]])

In [23]:
import pickle

pickle.dump(xgb ,open('modelxgb.pkl','wb'))
pickle.dump(cv,open('cv.pkl','wb'))